In [1]:
import random
import numpy as np

class card_tag:
    def __init__(self, cost = None, land = False, fetch = False, surveil = False, fetch_cost = None, land_type = None, can_fetch = None, protection = False, untapped = True, sac_target = False, sac_outlet = False, produces_green = False, produces_blue = False, green = False, pitch_cost = None, cantrip = False, effective_cantrip_strength = None,):
        self.cost = cost
        self.land = land
        self.fetch = fetch
        self.surveil = surveil
        self.fetch_cost = fetch_cost
        self.land_type = land_type
        self.can_fetch = can_fetch
        self.untapped = untapped
        self.protection = protection
        self.sac_target = sac_target
        self.sac_outlet = sac_outlet
        self.produces_green = produces_green
        self.produces_blue = produces_blue
        self.green = green
        self.pitch_cost = pitch_cost

CARD_TAGS = {
    "Allosaurus Rider" : card_tag(sac_target=True, green=True, pitch_cost="GG"),
    "Atraxa, Grand Unifier" : card_tag(green=True),
    "Boseiju, Who Endures" : card_tag(land=True, produces_green=True),
    "Breeding Pool" : card_tag(land=True, produces_green=True, produces_blue=True, land_type=["Forest", "Island"]),
    "Bridgeworks Battle" : card_tag(land=True, green=True, produces_green=True),
    "Consign to Memory" : card_tag(),
    "Disciple of Freyalise" : card_tag(land=True, green=True, produces_green=True),
    "Eldritch Evolution": card_tag(cost = "GGC", sac_outlet=True, green=True,),
    "Endurance": card_tag(green=True),
    "Flooded Strand": card_tag(land=True, fetch=True, can_fetch=['Island', 'Plain']),
    "Forest": card_tag(land=True, land_type=["Forest"], produces_green=True),
    "Generous Ent": card_tag(green=True, fetch=True, can_fetch=["Forest"], fetch_cost=1),
    "Ghalta, Stampede Tyrant": card_tag(green=True,),
    "Griselbrand": card_tag(),
    "Hedge Maze": card_tag(land=True, untapped=False, produces_green=True, produces_blue=True, land_type=['Island', "Forest"], surveil=True,),
    "Hooting Mandrills": card_tag(green=True), # for now I'll ignore mandrills, I'll implement delve later
    "Island" : card_tag(land=True, land_type=['Island'], produces_green=True),
    "Misty Rainforest": card_tag(land=True, fetch=True, can_fetch=['Island', 'Forest']),
    "Neoform": card_tag(cost = "UG", sac_outlet=True, green=True,),
    "Pact of Negation" : card_tag(protection=True),
    "Planar Genesis": card_tag(cost = "UG", green=True, cantrip=True, effective_cantrip_strength=4),
    "Preordain": card_tag(cost = "U", cantrip=True, effective_cantrip_strength=3),
    "Scalding Tarn": card_tag(land=True, fetch=True, can_fetch=['Island', 'Mountain']),
    "Summoner's Pact": card_tag(sac_target=True, green=True, pitch_cost="GG"), # for now we'll just consider pact as additional copies of Rider
    "Veil of Summer": card_tag(cost = "G", green=True, protection=True),
    "Xenagos, God of Revels": card_tag(green=True)
}

In [2]:
deck_list = {
    "Allosaurus Rider" : 4,
    "Atraxa, Grand Unifier" : 1,
    "Boseiju, Who Endures" : 1,
    "Breeding Pool" : 1,
    "Bridgeworks Battle" : 2,
    "Consign to Memory" : 4,
    "Disciple of Freyalise" : 2,
    "Eldritch Evolution": 4,
    "Endurance": 1,
    "Flooded Strand": 2,
    "Forest": 1,
    "Generous Ent": 2,
    "Ghalta, Stampede Tyrant": 2,
    "Griselbrand": 1,
    "Hedge Maze": 3,
    "Hooting Mandrills": 1, # for now I'll ignore mandrills, I'll implement delve later
    "Island" : 1,
    "Misty Rainforest": 4,
    "Neoform": 4,
    "Pact of Negation" : 4,
    "Planar Genesis": 4,
    "Preordain": 0,
    "Scalding Tarn": 2,
    "Summoner's Pact": 4, # for now we'll just consider pact as additional copies of Rider
    "Veil of Summer": 4,
    "Xenagos, God of Revels": 1
}

def get_starting_hand(deck):
    return deck[:7]

def initialize_deck(deck_list):
    deck = []

    for key, value in deck_list.items():
        deck.extend([key]*value)

    random.shuffle(deck)
    deck_card_tags = [CARD_TAGS[card] for card in deck]

    full_deck = [(name, tags) for name, tags in zip(deck, deck_card_tags)]

    return full_deck

def draw(deck, n):
    drawn_cards = []
    for i in range(n):
        drawn_cards.append(deck.pop())

    return drawn_cards

def get_card_names(card_list):
    return [card[0] for card in card_list]

def check_if_castable(card, mana_pool):
    temp_mana_pool = mana_pool.copy()
    cost = card[1].cost
    for pip in cost:
        if pip != "C":
            if temp_mana_pool[pip] > 0:
                temp_mana_pool[pip] -= 1
            else:
                return False
        else:
            mana_available = sum([value for _, value in temp_mana_pool.items()])
            if mana_available > 0:
                if temp_mana_pool['G'] > 0:
                    temp_mana_pool['G'] -= 1
                elif temp_mana_pool['U'] > 0:
                    temp_mana_pool['U'] -= 1    
            else:
                return False
    return True

def get_missing_mana(card, mana_pool):
    temp_mana_pool = mana_pool.copy()
    cost = card[1].cost
    missing_mana = ""
    for pip in cost:
        if pip != "C":
            if temp_mana_pool[pip] > 0:
                temp_mana_pool[pip] -= 1
            else:
                missing_mana += pip
        else:
            mana_available = sum([value for _, value in temp_mana_pool.items()])
            if mana_available > 0:
                if temp_mana_pool['G'] > 0:
                    temp_mana_pool['G'] -= 1
                elif temp_mana_pool['U'] > 0:
                    temp_mana_pool['U'] -= 1    
            else:
                missing_mana += "C"
    return missing_mana
def simulate_game_on_play_no_mull(deck_list, turns_to_win = 50):
    deck = initialize_deck(deck_list)
    hand = draw(deck, n=7)  # draw() returns a list of tuples, so hand is a list of tuples
    battlefield = []
    mana_production = {"U": 0, "G": 0}

    for turn in range(1, turns_to_win+1):
        if turn != 1:
            # draw a card and extend the hand (don't append the list)
            drawn = draw(deck, 1)
            if drawn:
                hand.extend(drawn)

        mana_pool = mana_production
        sac_targets = [card for card in hand if card[1].sac_target]
        sac_outlets = [card for card in hand if card[1].sac_outlet]
        green_cards = [card for card in hand if card[1].green]
        sac_outlet_castable = np.any([check_if_castable(sac_outlet, mana_pool) for sac_outlet in sac_outlets])
        sac_target_castable = (len(sac_targets) >= 0) & (len(green_cards) >= 4) # you need to take the rider and the sac outlet into account
        win = sac_outlet_castable & sac_target_castable
        if win:
            return turn
        
        missing_mana_to_cast_sac_outlet = [get_missing_mana(sac_outlet, mana_pool) for sac_outlet in sac_outlets]
        # play a land (choose the first land in hand)
        lands_in_hand = [card for card in hand if card[1].land]
        if lands_in_hand:
            # priority for playing lands should be fetch -> surveil if missing combo pieces -> duals -> the right land to fix our mana
            land = lands_in_hand[0]
            hand.remove(land)

            if land[1].fetch:
                if len(sac_targets)== 0 or len(sac_outlets)==0:
                    surveil_lands_in_deck = [card for card in deck if card[1].surveil]
                    if surveil_lands_in_deck:
                        fetched_land = surveil_lands_in_deck[0]
                    else:
                        fetchable_types = land[1].can_fetch
                        lands_with_types = [card for card in deck if card[1].land_type]
                        fetchable_lands = [card for card in lands_with_types if np.any([land_type in fetchable_types for land_type in card[1].land_type])]
                        if fetchable_lands:
                            fetched_land = fetchable_lands[0]
                        else:
                            fetched_land = land
                elif len(green_cards) < 4:
                    surveil_lands_in_deck = [card for card in deck if card[1].surveil]
                    if surveil_lands_in_deck:
                        fetched_land = surveil_lands_in_deck[0]
                    else:
                        fetchable_types = land[1].can_fetch
                        lands_with_types = [card for card in deck if card[1].land_type]
                        fetchable_lands = [card for card in lands_with_types if np.any([land_type in fetchable_types for land_type in card[1].land_type])]
                        if fetchable_lands:
                            fetched_land = fetchable_lands[0]
                        else:
                            fetched_land = land
                else:
                    fetchable_types = land[1].can_fetch
                    lands_with_types = [card for card in deck if card[1].land_type]
                    fetchable_lands = [card for card in lands_with_types if np.any([land_type in fetchable_types for land_type in card[1].land_type])]
                    if fetchable_lands:
                            fetched_land = fetchable_lands[0]
                    else:
                        fetched_land = land
                battlefield.append(fetched_land)
            else:
                battlefield.append(land)
            # update mana production list for simple bookkeeping
            if getattr(battlefield[-1][1], "produces_green", False):
                mana_production['G'] += 1
            if getattr(battlefield[-1][1], "produces_blue", False):
                mana_production['U'] += 1
            
        mana_pool = mana_production
        sac_targets = [card for card in hand if card[1].sac_target]
        sac_outlets = [card for card in hand if card[1].sac_outlet]
        green_cards = [card for card in hand if card[1].green]
        sac_outlet_castable = np.any([check_if_castable(sac_outlet, mana_pool) for sac_outlet in sac_outlets])
        sac_target_castable = (len(sac_targets) >= 0) & (len(green_cards) >= 4) # you need to take the rider and the sac outlet into account
        win = sac_outlet_castable & sac_target_castable
        if win:
            return turn

    return None
    
win = []
for i in range(50000):
    w = simulate_game_on_play_no_mull(deck_list)
    win.append(w)

In [3]:
win = np.array(win, dtype=float)
average_turn_win = np.mean(win)
median_turn_win = np.median(win)

print(f'This list wins on average on turn : {average_turn_win}, median : {median_turn_win}')

This list wins on average on turn : 4.53888, median : 3.0
